In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from pulp import *

import os
import openpyxl
from datetime import datetime, timedelta
import locale
folder_path = "Data"
file_path = "Modèle_Belfaa_updated_linearized_1303_simul_noQ.xlsx"
class DataProcessor:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.df_price = None
        self.df_chargesvar = None
        self.df_prod = None
        self.df_plantation = None
        self.df_month_index = None
        self.df_sim = None
        
        self.variety_scenario_dict = None
        self.scenario_variety_mapping = None
        self.serre_sau_dict = None
        self.scenario_prod = None
        self.month_to_week_indices = None
        self.scenario_cmo = None
        self.scenario_vitesse = None
        self.scenario_cv = None
        self.scenario_delai_dict = None
        self.scenario_duree_dict = None
        self.scenario_mois_dict = None
        self.scenario_culture = None
        self.secteur_serre_dict = None
        self.serre_secteur_dict = None
        self.price = None
        self.scenarios = None
        self.num_serre = None
        self.num_sect = None
        self.prod = None
        self.prod_mat = None
        self.serre_sau_dict=None
        self.scenario_couple=None
    
    def get_data(self):
        self.df_price = pd.read_csv(os.path.join(self.folder_path, "Prices.csv")).fillna(0)
        self.df_chargesvar = pd.read_csv(os.path.join(self.folder_path, "Charges_var.csv"))
        self.df_prod = pd.read_csv(os.path.join(self.folder_path, "Production.csv")).fillna(0)
        self.df_plantation = pd.read_csv(os.path.join(self.folder_path, "plantation.csv"))
        self.df_month_index = pd.read_csv(os.path.join(self.folder_path, "month_index.csv"))
        self.df_sim = pd.read_csv(os.path.join(self.folder_path, "Simulation.csv"))
        
    def get_dict(self):
        serre_sau_dict = self.df_sim.set_index('serre')['SAU(ha)'].to_dict()
        self.serre_sau_dict=serre_sau_dict
        scenario_variety_mapping = self.df_prod[['Scénario', 'variété 23-24']]
        sum_last_37 = self.df_prod.iloc[:, -37:].sum(axis=1)
        variety_scenario_dict = {}
        scenario_prod = dict(zip(self.df_prod['Scénario'], sum_last_37))
        
        for index, row in scenario_variety_mapping.iterrows():
            scenario = row['Scénario']
            variety = row['variété 23-24']
            if variety not in variety_scenario_dict:
                variety_scenario_dict[variety] = []
            variety_scenario_dict[variety].append(scenario)
        
        self.variety_scenario_dict = variety_scenario_dict
        self.scenario_variety_mapping = scenario_variety_mapping
        self.serre_sau_dict = serre_sau_dict
        self.scenario_prod = scenario_prod
        
    def month_week_dict(self):
        locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')
        start_date = datetime(2024, 1, 1)
        data = []
        months = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Aout', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
        num_weeks = 84
        
        for index in range(1, num_weeks + 1):
            month_index = start_date.month
            month_name = months[month_index - 1]
            month = month_name.capitalize()
            month_index = self.get_month_index(month)
            data.append([month, start_date.strftime('%d/%m/%Y'), month_index, index])
            start_date += timedelta(weeks=1)
        
        df_week_index = pd.DataFrame(data, columns=['Month', 'Date', 'Month Index', 'Week Index'])
        month_to_week_indices = {}
        
        for index, row in df_week_index.iterrows():
            month = row['Month']
            week_index = row['Week Index']
            if month not in month_to_week_indices:
                month_to_week_indices[month] = []
            month_to_week_indices[month].append(week_index)
        
        self.month_to_week_indices = month_to_week_indices
    
    def get_month_index(self, month):
        month_dict = {
            'Janvier': 1,
            'Février': 2,
            'Mars': 3,
            'Avril': 4,
            'Mai': 5,
            'Juin': 6,
            'Juillet': 7,
            'Aout': 8,
            'Septembre': 9,
            'Octobre': 10,
            'Novembre': 11,
            'Décembre': 12
        }
        return month_dict[month]
    
    def extract_scenario_charges(self):
        scenario_cmo = {}
        scenario_vitesse = {}
        scenario_cv = {}
        
        for index, row in self.df_chargesvar.iterrows():
            scenario = row['Scénario']
            cmo = row["Coûts de Main d'œuvre par kg de production"]
            vitesse = row["Vitesse de main d'œuvre kg/personne/jour"]
            cv = row["Coûts variable par hectar"]
            scenario_cmo[scenario] = cmo
            scenario_vitesse[scenario] = vitesse
            scenario_cv[scenario] = cv
            
        self.scenario_cmo = scenario_cmo
        self.scenario_vitesse = scenario_vitesse
        self.scenario_cv = scenario_cv
    
    def extract_scenario_production(self):
        scenario_delai_dict = {}
        scenario_duree_dict = {}
        scenario_mois_dict = {}
        scenario_culture = {}
        scenario_couple={}
        for index, row in self.df_prod.iterrows():
            scenario = row['Scénario']
            delai = row['Délai pour début de production']
            duree = row['Durée de production en semaine']
            mois = row['Mois']
            couple=row['Couple']+" "+row["Type de plantations 23-24"]
            culture = row['Culture']
            
            scenario_delai_dict[scenario] = delai
            scenario_duree_dict[scenario] = duree
            scenario_mois_dict[scenario] = mois
            scenario_culture[scenario] = culture
            scenario_couple[scenario]=couple
        
        self.scenario_delai_dict = scenario_delai_dict
        self.scenario_duree_dict = scenario_duree_dict
        self.scenario_mois_dict = scenario_mois_dict
        self.scenario_culture = scenario_culture
        self.scenario_couple=scenario_couple
    def extract_sim_data(self):
        secteur_serre_dict = {}
        
        for index, row in self.df_sim.iterrows():
            secteur = row['secteur']
            serre = row['serre']
            if secteur not in secteur_serre_dict:
                secteur_serre_dict[int(secteur)] = [int(serre)]
            else:
                secteur_serre_dict[int(secteur)].append(int(serre))
        
        serre_secteur_dict = {serre: secteur for secteur, serres in secteur_serre_dict.items() for serre in serres}
        
        self.secteur_serre_dict = secteur_serre_dict
        self.serre_secteur_dict = serre_secteur_dict
    
    def extract_price_data(self):
        price = {}
        price["Framboise"] = np.array(self.df_price.iloc[0, 1:])
        price["Mure"] = np.array(self.df_price.iloc[1, 1:])
        price["Adelita"] = np.array(self.df_price.iloc[2, 1:])
        self.price = price
    
    def other_data(self):
        scenarios = list(self.scenario_culture.keys())
        num_serre = self.df_sim.shape[0]
        num_sect = len(self.secteur_serre_dict)
        
        self.scenarios = scenarios
        self.num_serre = num_serre
        self.num_sect = num_sect
    
    def padded_dot(self, a, b):
        len_a = a.shape[1]
        len_b = b.shape[1]
        max_len = max(len_a, len_b)
        a_padded = np.pad(a, ((0, 0), (0, max_len - len_a)), mode='constant', constant_values=0)
        b_padded = np.pad(b, ((0, 0), (0, max_len - len_b)), mode='constant', constant_values=0)
        result = np.dot(a_padded, b_padded.T)
        return result
    
    def compute_tensor(self):
        prod_mat = np.matrix(self.df_prod.iloc[:, 8:])
        prod = {}
        
        for i in range(self.num_serre):
            for j in self.scenarios:
                for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                    if j in self.variety_scenario_dict["Adelita"]:
                        price_array = np.array(self.price["Adelita"][t-1 + self.scenario_delai_dict[j] :])
                        prod_mat_array = np.array(prod_mat[self.scenarios.index(j), :])
                        prod[(i, j, t)] = self.serre_sau_dict[i + 1] * self.padded_dot(
                            price_array.reshape(1, -1),
                            prod_mat_array.reshape(1, -1)
                        )
                    else:
                        price_array = np.array(self.price[self.scenario_culture[j]][t-1 + self.scenario_delai_dict[j]:])
                        prod_mat_array = np.array(prod_mat[self.scenarios.index(j), :])
                        prod[(i, j, t)] = self.serre_sau_dict[i + 1] * self.padded_dot(
                            price_array.reshape(1, -1),
                            prod_mat_array.reshape(1, -1)
                        )
        
        self.prod = prod
        self.prod_mat = prod_mat
    
class PortfolioModelPulp(DataProcessor):
    def __init__(self, data_processor):
        super().__init__(folder_path)  # Initialize the superclass (DataProcessor)
        self.__dict__.update(data_processor.__dict__)  # Inherit all variables from DataProcessor

    def optimize_portfolio(self):
  
        try:
            m = LpProblem("portfolio", LpMaximize)
            choices = LpVariable.dicts("choice", [(i, j, t) for i in range(self.num_serre)
                                          for j in self.scenarios
                                          for t in self.month_to_week_indices[self.scenario_mois_dict[j]]], 
                               cat=LpBinary)

            for i in range(self.num_serre):
                m += lpSum(choices[(i, j, t)] for j in self.scenarios
                   for t in self.month_to_week_indices[self.scenario_mois_dict[j]]) == 1

            for i in range(self.num_sect):
                ref = self.secteur_serre_dict[i + 1][0]
                for j in self.secteur_serre_dict[i + 1]:
                    for k in self.scenarios:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[k]]:
                            m += choices[(j - 1, k, t)] == choices[(ref - 1, k, t)]

            for i in range(21):
                for t in self.month_to_week_indices[self.scenario_mois_dict[5]]:
                    m += choices[(i, 5, t)] == 0
                for t in self.month_to_week_indices[self.scenario_mois_dict[4]]:
                    m += choices[(i, 4, t)] == 0

            for i in range(self.num_serre):
                if i != 19 and i != 20:
                    for j in self.variety_scenario_dict["Clara"]:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            m += choices[(i, j, t)] == 0
                    for j in self.variety_scenario_dict["LAURITA"]:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            m += choices[(i, j, t)] == 0
                else:
                    if self.serre_sau_dict[i + 1] > 2.87:
                        for j in self.variety_scenario_dict["Clara"]:
                            for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                                m += choices[(i, j, t)] == 0
                        for j in self.variety_scenario_dict["LAURITA"]:
                            for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                                m += choices[(i, j, t)] == 0

            for s in range(1, 91):
                scenarios_time_dict = {}
                for j in self.scenarios:
                    scenarios_time_dict[j] = []
                    if j in [4, 5, 20]:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            if t + self.scenario_delai_dict[j] < s and s < t + self.scenario_delai_dict[j] + 38:
                                scenarios_time_dict[j].append(t)
                    else:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            if t + self.scenario_delai_dict[j] < s and s < t + self.scenario_delai_dict[j] + self.scenario_duree_dict[j] + 1:
                                scenarios_time_dict[j].append(t)
                m += lpSum(choices[(i, k, t)] * (self.serre_sau_dict[i + 1] / self.scenario_vitesse[k]) *
                   self.prod_mat[self.scenarios.index(k), s - t - self.scenario_delai_dict[k] - 1]
                   for k in self.scenarios for t in scenarios_time_dict[k] if scenarios_time_dict[k] != []
                   for i in range(self.num_serre)) <= 7 * 600

            CA_expr = lpSum(choices[(i, j, t)] * self.prod[(i, j, t)]
                    for i in range(self.num_serre) for j in self.scenarios
                    for t in self.month_to_week_indices[self.scenario_mois_dict[j]])

            CMO_expr = lpSum(choices[(i, j, t)] * self.scenario_prod[j] * self.serre_sau_dict[i + 1] * self.scenario_cmo[j]
                     for j in self.scenarios for t in self.month_to_week_indices[self.scenario_mois_dict[j]]
                     for i in range(self.num_serre))

            CV_expr = lpSum(choices[(i, j, t)] * self.serre_sau_dict[i + 1] * self.scenario_cv[j]
                    for j in self.scenarios for t in self.month_to_week_indices[self.scenario_mois_dict[j]]
                    for i in range(self.num_serre))

            m += (CA_expr - CV_expr - CMO_expr)

            m.solve()
            CA_expr=value(CA_expr)[0][0]+value(CMO_expr)+ value(CV_expr)
            CMO_expr=value(CMO_expr)
            CV_expr=value(CV_expr)
            obj=CA_expr - CV_expr - CMO_expr
            print("Value of CA_expr:",CA_expr )
            print("Value of CMO_expr:", CMO_expr)
            print("Value of CV_expr:", CV_expr)
            print("Obj:", obj)

            scenario_chosen, semaines_chosen = [], []
            if LpStatus[m.status] == "Optimal":
        # Get selected variables with value == 1
                list_var = [v.name for v in m.variables() if v.varValue == 1]
                sorted_data = sorted(list_var, key=lambda x: (int(x.split(',')[0].split('(')[-1]), int(x.split(',')[1].split('_')[-1]), int(x.split(',')[2].split('_')[-1].rstrip(')'))))

            for i in sorted_data:
                 scenario_chosen.append(int(i.split("_")[2][:-1]))
                 semaines_chosen.append(int(i.split("_")[3][:-1]))
            
            def write_arrays_to_excel(filename, scenario_chosen, semaines_chosen):
                # Load the workbook
                wb = openpyxl.load_workbook(filename)
                # Select the worksheet named "Simulation"
                ws = wb["Simulation"]

                # Write scenario_chosen to column E starting from cell E23
                for i, value in enumerate(scenario_chosen):
                    ws.cell(row=23+i, column=5, value=value)

                # Write semaines_chosen to column M starting from cell M23
                for i, value in enumerate(semaines_chosen):
                    ws.cell(row=23+i, column=13, value=value)

                # Save the workbook
                wb.save(filename)
                print("All safe and sound")
            write_arrays_to_excel("Modèle_Belfaa_updated_linearized_1303_simul_noQ.xlsx", scenario_chosen, semaines_chosen)
       
        
        except Exception as e:
            print(e)


class PortfolioModelGurobi(DataProcessor):
    def __init__(self, data_processor):
        super().__init__(folder_path)  # Initialize the superclass (DataProcessor)
        self.__dict__.update(data_processor.__dict__)
        self.scenario_chosen=None
        self.semaines_chosen=None  # Inherit all variables from DataProcessor
    
    def optimize_portfolio(self):
        try:
            m = gp.Model("portfolio")
            choices = {}
            for i in range(self.num_serre):
                for j in self.scenarios:
                    for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                        choices[(i, j, t)] = m.addVar(vtype=GRB.BINARY, name=f'choice_{i}_{j}_{t}')

            for i in range(self.num_serre):
                m.addConstr(gp.quicksum(choices[(i, j, t)] for j in self.scenarios 
                                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]) == 1, f"constraint_{i}")

            for i in range(self.num_sect):
                ref = self.secteur_serre_dict[i+1][0]
                for j in self.secteur_serre_dict[i+1]:
                    for k in self.scenarios:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[k]]:
                            m.addConstr(choices[(j-1,k,t)] == choices[(ref-1,k,t)], f'c_0_{j}_{k}_{t}')

            for i in range(21):
                for t in self.month_to_week_indices[self.scenario_mois_dict[5]]:
                    m.addConstr(choices[(i,5,t)] == 0)
                for t in self.month_to_week_indices[self.scenario_mois_dict[4]]:
                    m.addConstr(choices[(i,4,t)] == 0)

            for i in range(self.num_serre):
                if i!=19 and i!=20:
                    for j in self.variety_scenario_dict["Clara"]:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            m.addConstr(choices[(i,j,t)] == 0)
                    for j in self.variety_scenario_dict["LAURITA"]:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            m.addConstr(choices[(i,j,t)] == 0)
                else:
                    if self.serre_sau_dict[i+1] > 2.87:
                        for j in self.variety_scenario_dict["Clara"]:
                            for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                                m.addConstr(choices[(i,j,t)] == 0)
                        for j in self.variety_scenario_dict["LAURITA"]:
                            for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                                m.addConstr(choices[(i,j,t)] == 0)

            for s in range(1, 91):
                scenarios_time_dict = {}
                for j in self.scenarios:
                    scenarios_time_dict[j] = []
                    if j in [4, 5, 20]:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            if t + self.scenario_delai_dict[j] < s and s < t + self.scenario_delai_dict[j] + 38:
                                scenarios_time_dict[j].append(t)
                    else:
                        for t in self.month_to_week_indices[self.scenario_mois_dict[j]]:
                            if t + self.scenario_delai_dict[j] < s and s < t + self.scenario_delai_dict[j] + self.scenario_duree_dict[j] + 1:
                                scenarios_time_dict[j].append(t)
                m.addConstr(gp.quicksum(choices[(i,k,t)]*(self.serre_sau_dict[i+1]/self.scenario_vitesse[k]) *
                                         self.prod_mat[self.scenarios.index(k),s-t-self.scenario_delai_dict[k]-1]
                                         for k in self.scenarios for t in scenarios_time_dict[k] if scenarios_time_dict[k]!=[]
                                         for i in range(self.num_serre)) <= 7*600, f'mo_{s}')

            CA_expr = gp.quicksum(choices[(i,j,t)] * self.prod[(i, j, t)] 
                                  for i in range(self.num_serre) for j in self.scenarios 
                                  for t in self.month_to_week_indices[self.scenario_mois_dict[j]])

            CMO_expr = gp.quicksum(choices[(i,j,t)] * self.scenario_prod[j] * self.serre_sau_dict[i+1] * self.scenario_cmo[j] 
                                   for j in self.scenarios for t in self.month_to_week_indices[self.scenario_mois_dict[j]]
                                   for i in range(self.num_serre))

            CV_expr = gp.quicksum(choices[(i,j,t)] * self.serre_sau_dict[i+1] * self.scenario_cv[j] 
                                  for j in self.scenarios for t in self.month_to_week_indices[self.scenario_mois_dict[j]]
                                  for i in range(self.num_serre))

            m.update()
            m.setObjective((CA_expr - CV_expr - CMO_expr), GRB.MAXIMIZE)
            m.optimize()

            print("Value of CA_expr:", CA_expr.getValue())
            print("Value of CMO_expr:", CMO_expr.getValue())
            print("Value of CV_expr:", CV_expr.getValue())
            print(f"Obj: {m.ObjVal:g}")
            # Your optimization code here, accessing variables inherited from DataProcessor
            scenario_chosen, semaines_chosen = [], []
            if m.status == GRB.Status.OPTIMAL:
                # Get selected variables with value == 1
                list_var = [v.varName for v in m.getVars() if v.x == 1]

            for i in list_var:
                scenario_chosen.append(int(i.split("_")[2]))
                semaines_chosen.append(int(i.split("_")[3]))
            self.scenario_chosen=scenario_chosen
            self.semaines_chosen=semaines_chosen
            
            

    

        except gp.GurobiError as e:
            print(f"Error code {e.errno}: {e}")
        except AttributeError:
            print("Encountered an attribute error")
            
        



In [2]:
folder_path = "Data"
file_path = "Modèle_Belfaa_updated_linearized_1303_simul_noQ.xlsx"

In [3]:
data_processor = DataProcessor(folder_path)
data_processor.get_data()
data_processor.get_dict()
data_processor.month_week_dict()
data_processor.extract_scenario_charges()
data_processor.extract_scenario_production()
data_processor.extract_sim_data()
data_processor.extract_price_data()
data_processor.other_data()
data_processor.compute_tensor()
    


portfolio_model = PortfolioModelGurobi(data_processor)


In [4]:
portfolio_model.optimize_portfolio()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3103 rows, 2280 columns and 28057 nonzeros
Model fingerprint: 0xd6c832b6
Variable types: 0 continuous, 2280 integer (2280 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [1e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective -5933765.250
Presolve removed 3066 rows and 2025 columns
Presolve time: 0.04s
Presolved: 37 rows, 255 columns, 3544 nonzeros
Found heuristic solution: objective 1.042128e+07
Variable types: 0 continuous, 255 integer (255 binary)

Root relaxation: objective 1.227904e+07, 71 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  

In [14]:
from datetime import datetime, timedelta

# Define the start date
start_date = datetime(2024, 1, 1)

# Dictionary of French month names
french_months = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}

# Function to get the month from the week index
def get_month_from_week_index(week_index):
    # Calculate the date corresponding to the week index
    target_date = start_date + timedelta(weeks=week_index - 1)
    
    # Get the month index from the target date
    month_index = target_date.month
    
    # Get the French month name from the month index using the dictionary
    month_name = french_months[month_index]
    
    return month_name

data_dict = portfolio_model.scenario_variety_mapping.to_dict(orient='records')
data_dict = {entry['Scénario']: entry['variété 23-24'] for entry in data_dict}

data = {
            "Secteurr": list(portfolio_model.serre_secteur_dict[i] for i in range(1, portfolio_model.num_serre+1)),
            "Serre": list(range(1, portfolio_model.num_serre + 1)),
            "Sau": list(portfolio_model.serre_sau_dict.values()),
            "Scenario_index": portfolio_model.scenario_chosen,
            "Variété": [data_dict[scenario] for scenario in portfolio_model.scenario_chosen],
            "Scenario":[portfolio_model.scenario_couple[i] for i in portfolio_model.scenario_chosen],
            "Scenario Mois":[get_month_from_week_index(i) for i in portfolio_model.semaines_chosen],
            "Cout de la main d'oeuvre": [portfolio_model.scenario_cmo[i] for i in portfolio_model.scenario_chosen],
            "Cout variables": [portfolio_model.scenario_cv[i] for i in portfolio_model.scenario_chosen],
            "Vitesse de récolte": [portfolio_model.scenario_vitesse[i] for i in portfolio_model.scenario_chosen],
            
            "Week of plantation": portfolio_model.semaines_chosen,
            
            
            "Weeks to debut plantation":[portfolio_model.scenario_delai_dict[i] for i in portfolio_model.scenario_chosen]

            }
df = pd.DataFrame(data)
df


,Secteurr,Serre,Sau,Scenario_index,Variété,Scenario,Scenario Mois,Cout de la main d'oeuvre,Cout variables,Vitesse de récolte,Week of plantation,Weeks to debut plantation
0,1,1,1.10,3,YAZMIN,YAZMIN-TP TP,Août,7.33,632256.0,15,35,15
1,1,2,1.10,3,YAZMIN,YAZMIN-TP TP,Août,7.33,632256.0,15,35,15
2,1,3,1.10,3,YAZMIN,YAZMIN-TP TP,Août,7.33,632256.0,15,35,15
3,1,4,1.10,3,YAZMIN,YAZMIN-TP TP,Août,7.33,632256.0,15,35,15
4,1,5,1.65,3,YAZMIN,YAZMIN-TP TP,Août,7.33,632256.0,15,35,15
5,1,6,1.19,3,YAZMIN,YAZMIN-TP TP,Août,7.33,632256.0,15,35,15
6,2,7,1.48,11,VICTORIA,VICTORIA-Long canne ST Long canne ST,Novembre,7.86,925440.0,20,45,13
7,2,8,1.41,11,VICTORIA,VICTORIA-Long canne ST Long canne ST,Novembre,7.86,925440.0,20,45,13
8,2,9,1.29,11,VICTORIA,VICTORIA-Long canne ST Long canne ST,Novembre,7.86,925440.0,20,45,13
9,2,10,1.29,11,VICTORIA,VICTORIA-Long canne ST Long canne ST,Novembre,7.86,925440.0,20,45,13


{1: 'Framboise',
 2: 'Framboise',
 3: 'Framboise',
 4: 'Framboise',
 5: 'Framboise',
 8: 'Framboise',
 9: 'Framboise',
 10: 'Framboise',
 11: 'Mure',
 12: 'Mure',
 13: 'Mure',
 14: 'Mure',
 15: 'Framboise',
 18: 'Mure',
 19: 'Mure'}

In [ ]:

print(data_dict)

{1: 'YAZMIN', 2: 'YAZMIN', 3: 'YAZMIN', 4: 'YAZMIN', 5: 'YAZMIN', 8: 'Adelita', 9: 'Adelita', 10: 'Adelita', 11: 'VICTORIA', 12: 'LAURITA', 13: 'LAURITA', 14: 'LAURITA', 15: 'YAZMIN', 18: 'Clara', 19: 'Henrienta'}


In [ ]:


print()

YAZMIN
